In [147]:
import re

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [148]:
 spark = SparkSession\
        .builder\
        .appName("TwitterSentimentAnalysis")\
        .master("local[*]")\
        .getOrCreate()

In [149]:
spark.sparkContext.setLogLevel('WARN')

In [150]:
lines = spark.readStream\
        .format("socket")\
        .option("host", "127.0.0.1")\
        .option("port", 3333)\
        .load()

22/11/18 01:08:33 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [151]:
tweet_df_string = lines.selectExpr("CAST(value AS STRING)")

tweets_tab = tweet_df_string.withColumn('word', explode(split(col('value'), ' '))) \
    .groupBy('word') \
    .count() \
    .sort('count', ascending=False). \
    filter(col('word').contains('#'))

In [152]:
writeTweet = tweets_tab.writeStream. \
    outputMode("complete"). \
    format("memory"). \
    queryName("tweetquery"). \
    trigger(processingTime='2 seconds'). \
    start()

22/11/18 01:08:38 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/vw/8j0kbq6n5z7682q0r894_vxm0000gn/T/temporary-fd0e6fbf-062a-4c41-8a48-45afcc7d14ae. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/11/18 01:08:38 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


22/11/18 01:08:48 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 9607 milliseconds
22/11/18 01:09:10 WARN TextSocketMicroBatchStream: Stream closed by 127.0.0.1:3333


22/11/18 01:09:22 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 10899 milliseconds


In [146]:
spark.stop()
